In [1]:
import pandas as pd
import ast
import re
import unidecode

# Constantes y patrones
UNITS = r"g|gr|grs|grams|gramos|kg|ml|mls|cl|l|cc"
COUNTABLE_UNITS = [
    'ou', 'ous',
    'all', 'alls',
    'llesca', 'llesques',
    'dent', 'dents',
    'cullerada', 'cullerades',
    'manat', 'manats',
    'blanc', 'blancs',
    'gra', 'grans',
    'tros', 'trossos',
    'tassa', 'tasses',
    'unitat', 'unitats',
    'passig', 'passigs',
    'tall', 'talls',
    'fulla', 'fulles',
    'got', 'gots',
    'punta', 'puntes',
    'branca', 'branques',
    'raig', 'raigs',
    'pessic', 'pessics',
    'llauna', 'llaunes',
    'paquet', 'paquets'
]

def clean_parentheses(txt):
    """Elimina todo el contenido entre paréntesis y los paréntesis"""
    return re.sub(r'\([^)]*\)', '', txt).strip()

def clean_text(txt):
    """Limpia y normaliza el texto"""
    txt = clean_parentheses(txt)
    txt = unidecode.unidecode(txt.lower()).strip()
    txt = re.sub(r'\s+', ' ', txt)
    return txt.strip()

def parse_fraction(s):
    """Convierte fracciones tipo '1/2' o '½' a float"""
    s = s.replace('½', '1/2').replace('⁄', '/')
    if '/' in s:
        try:
            num, den = s.split('/')
            return float(num) / float(den)
        except:
            return None
    try:
        return float(s)
    except:
        return None

def parse_ingredient(ing):
    try:
        txt = clean_text(ing)

        # 1. Ingredientes "al gust"
        if any(x in txt for x in ['al gust', 'al gusto', 'a gust']):
            nombre = txt.replace('al gust', '').replace('al gusto', '').replace('a gust', '').strip()
            return None, None, nombre

        # 2. Rangos en cualquier parte: busca "100-150g", "10-20 g", etc.
        pattern_anywhere_range = r'(\d+(?:[.,]\d+)?)[\s\-–]+(\d+(?:[.,]\d+)?)(?:\s*)(g|gr|grs|grams|gramos|kg|ml|mls|cl|l|cc)?'
        match = re.search(pattern_anywhere_range, txt)
        if match:
            cantidad = match.group(2).replace(',', '.')
            unidad = match.group(3)
            if unidad in ['gr', 'grs', 'grams', 'gramos']:
                unidad = 'g'
            # Eliminar el rango y la unidad del nombre
            nombre = re.sub(pattern_anywhere_range, '', txt).strip()
            return float(cantidad), unidad, nombre

        # 3. Fracciones: "1/2 alvocat", "½ ceba"
        pattern_frac = r'^(?P<frac>\d+/\d+|½)\s*(?P<unit>'+r'|'.join(COUNTABLE_UNITS)+r')?(?:\s+de\s+|\s+d\'|\s+)?(?P<name>.+)?'
        match = re.match(pattern_frac, txt)
        if match:
            cantidad = parse_fraction(match.group('frac'))
            unidad = match.group('unit')
            nombre = match.group('name') if match.group('name') else (unidad if unidad else txt)
            if unidad and match.group('name'):
                nombre = f"{unidad} de {nombre}".strip()
                unidad = None
            return cantidad, unidad, nombre.strip()

        # 4. Cantidad + unidad: "100g de patates", "10 ml d'oli"
        pattern_weight = r'^(?P<qty>\d+(?:[.,]\d+)?)\s*(?P<unit>'+UNITS+r')\s+(?:de\s+|d\'|\s+)?(?P<name>.+)$'
        match = re.match(pattern_weight, txt)
        if match:
            cantidad = float(match.group('qty').replace(',', '.'))
            unidad = match.group('unit')
            nombre = match.group('name')
            if unidad in ['gr', 'grs', 'grams', 'gramos']:
                unidad = 'g'
            return cantidad, unidad, nombre.strip()

        # 5. Cantidad + unidad contable: "2 ous", "1 passig de sesam"
        pattern_countable = r'^(?P<qty>\d+(?:[.,]\d+)?)\s*(?P<unit>'+r'|'.join(COUNTABLE_UNITS)+r')?(?:\s+de\s+|\s+d\'|\s+)?(?P<name>.+)?'
        match = re.match(pattern_countable, txt)
        if match:
            cantidad = float(match.group('qty').replace(',', '.'))
            unidad = match.group('unit')
            nombre = match.group('name') if match.group('name') else (unidad if unidad else txt)
            if unidad and match.group('name'):
                nombre = f"{unidad} de {nombre}".strip()
                unidad = None
            return cantidad, unidad, nombre.strip()

        # 6. "Ingrediente XXg": "patates 100g"
        pattern_reverse = r'^(?P<name>.+?)\s+(?P<qty>\d+(?:[.,]\d+)?)\s*(?P<unit>'+UNITS+r')$'
        match = re.match(pattern_reverse, txt)
        if match:
            cantidad = float(match.group('qty').replace(',', '.'))
            unidad = match.group('unit')
            nombre = match.group('name')
            if unidad in ['gr', 'grs', 'grams', 'gramos']:
                unidad = 'g'
            return cantidad, unidad, nombre.strip()

        # 7. Fracción sola: "1/2 alvocat"
        pattern_frac2 = r'^(?P<frac>\d+/\d+|½)\s+(?P<name>.+)$'
        match = re.match(pattern_frac2, txt)
        if match:
            cantidad = parse_fraction(match.group('frac'))
            nombre = match.group('name')
            return cantidad, None, nombre.strip()

        # 8. Si no coincide con ningún patrón, devolver el texto limpio
        return None, None, txt

    except Exception as e:
        print(f"Error procesando '{ing}': {str(e)}")
        return None, None, txt

def create_ingredients_dataset(input_file):
    """
    Crea el dataset de ingredientes a partir del archivo de recetas

    Args:
        input_file (str): Ruta al archivo CSV de recetas

    Returns:
        pd.DataFrame: DataFrame con los ingredientes procesados
    """
    # Cargar el CSV original
    print("Cargando archivo de recetas...")
    df = pd.read_csv(input_file)

    # Procesar ingredientes
    print("Procesando ingredientes...")
    rows = []
    for idx, row in df.iterrows():
        receta = row["nom"]
        try:
            ingredientes = ast.literal_eval(row["ingredients"])
            for ing in ingredientes:
                cantidad, unidad, nombre = parse_ingredient(ing)
                rows.append({
                    "receta": receta,
                    "ingrediente_original": ing,
                    "cantidad": cantidad,
                    "unidad": unidad,
                    "ingrediente_estandar": nombre
                })
        except Exception as e:
            print(f"Error en receta {receta}: {str(e)}")
            continue

    # Crear y retornar DataFrame con los resultados
    print("Creando DataFrame con resultados...")
    return pd.DataFrame(rows)

# Ejemplo de uso:
ingredientes_df = create_ingredients_dataset("recetas_cat_0301.csv")

# Mostrar algunos ejemplos de verificación
print("\nEjemplos de verificación:")
ejemplos = [
    "2 alls",
    "4 ous",
    "3 llesques de pernil salat",
    "1 manat de julivert",
    "2 blancs d'ou",
    "150-200 g de bledes",
    "100g de patates",
    "tomàquet 200g",
    "1 cullerada de mantega",
    "2 dents d'all",
    "3 grans d'all",
    "2 tasses de llet",
    "½ alvocat",
    "1/2 ceba",
    "2-3 talls de pernil",
    "1 passig de sesam"
]

print("\nVerificación de casos específicos:")
for ejemplo in ejemplos:
    cant, uni, nom = parse_ingredient(ejemplo)
    print(f"\nOriginal: {ejemplo}")
    print(f"Procesado: cantidad={cant}, unidad={uni}, nombre={nom}")

# Mostrar las primeras filas del dataset
print("\nPrimeras filas del dataset:")
print(ingredientes_df.head())

Cargando archivo de recetas...
Procesando ingredientes...
Creando DataFrame con resultados...

Ejemplos de verificación:

Verificación de casos específicos:

Original: 2 alls
Procesado: cantidad=2.0, unidad=None, nombre=all de s

Original: 4 ous
Procesado: cantidad=4.0, unidad=None, nombre=ou de s

Original: 3 llesques de pernil salat
Procesado: cantidad=3.0, unidad=None, nombre=llesques de pernil salat

Original: 1 manat de julivert
Procesado: cantidad=1.0, unidad=None, nombre=manat de julivert

Original: 2 blancs d'ou
Procesado: cantidad=2.0, unidad=None, nombre=blanc de s d'ou

Original: 150-200 g de bledes
Procesado: cantidad=200.0, unidad=g, nombre=de bledes

Original: 100g de patates
Procesado: cantidad=100.0, unidad=g, nombre=patates

Original: tomàquet 200g
Procesado: cantidad=200.0, unidad=g, nombre=tomaquet

Original: 1 cullerada de mantega
Procesado: cantidad=1.0, unidad=None, nombre=cullerada de mantega

Original: 2 dents d'all
Procesado: cantidad=2.0, unidad=None, nombre=d

In [53]:
ingredientes_df

,receta,ingrediente_original,cantidad,unidad,ingrediente_estandar
0,Iogurt grec amb fraules,Iogurt grec 200g,200.0,g,iogurt grec
1,Iogurt grec amb fraules,Fraules 100-150g,150.0,g,fraules
2,Iogurt grec amb fraules,Nous i ametlles 30g,30.0,g,nous i ametlles
3,Iogurt grec amb fraules,Xía 10-20g,20.0,g,xia
4,Iogurt grec amb fraules,Canyella al gust,NaN,None,canyella
...,...,...,...,...,...
1058,Pollastre al forn a les fines herbes,5g de romaní fresc (1g en pols),5.0,g,romani fresc
1059,Pollastre al forn a les fines herbes,5g d'orenga fresc (0.5g en pols),5.0,g,orenga fresc
1060,Pollastre al forn a les fines herbes,15g de mostassa Dijon,15.0,g,mostassa dijon
1061,Pollastre al forn a les fines herbes,15ml d'oli d'oliva (una cullerada),15.0,ml,oli d'oliva


In [3]:
ingredientes_df.to_csv("ingredientes_estandarizados.csv", index=False)

In [67]:
ingredientes_df.at[338, 'unidad'] = "g"

In [62]:
ingredientes_df.at[204, 'ingrediente_estandar'] = "cibulet"
ingredientes_df.at[204, 'cantidad'] = 5
ingredientes_df.at[204, 'unidad'] = "g"

In [49]:
ingredientes_df.loc[[85, 90, 115], 'ingrediente_estandar'] = "alls"

In [68]:
ingredientes_df[ingredientes_df["unidad"].str.contains("gr", na=False)]

,receta,ingrediente_original,cantidad,unidad,ingrediente_estandar
